# Decision Tree

In [105]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, _tree
from collections import defaultdict
from tqdm import tqdm

In [ ]:
# -----------------------------
# 1️⃣ Load and preprocess dataset
# -----------------------------
df = pd.read_csv("/content/drive/MyDrive/IACD/3 ANO/IC/adult.csv")

TARGET_COL = "income"
df = df.replace("?", np.nan).dropna()

X = df.drop(columns=TARGET_COL)
y = df[TARGET_COL]

categorical_features = X.select_dtypes(include="object").columns
numerical_features = X.select_dtypes(exclude="object").columns

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
    ("num", "passthrough", numerical_features)
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# -----------------------------
# 2️⃣ Train Decision Tree
# -----------------------------
tree_model = Pipeline([
    ("preprocessing", preprocessor),
    ("model", DecisionTreeClassifier(max_leaf_nodes=2048, random_state=42))
])

tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)

# Transform test set once
X_test_transformed = tree_model.named_steps["preprocessing"].transform(X_test)
feature_names = list(tree_model.named_steps["preprocessing"].get_feature_names_out())

In [ ]:
# -----------------------------
# 3️⃣ Subsample for fast evaluation
# -----------------------------
sample_size = 500  # adjust to keep runtime low
idx = np.random.choice(X_test_transformed.shape[0], size=sample_size, replace=False)
X_sub = X_test_transformed[idx]
y_sub = y_pred[idx]

In [ ]:
# -----------------------------
# 4️⃣ Extract leaf path explanations
# -----------------------------
def get_leaf_paths(tree, feature_names):
    paths = {}
    tree_ = tree.tree_
    def recurse(node, path):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            f_name = feature_names[tree_.feature[node]]
            threshold = tree_.threshold[node]
            recurse(tree_.children_left[node], path + [(f_name, "<=", threshold)])
            recurse(tree_.children_right[node], path + [(f_name, ">", threshold)])
        else:
            paths[node] = path
    recurse(0, [])
    return paths

paths = get_leaf_paths(tree_model.named_steps["model"], feature_names)

def get_instance_explanation(tree, X, paths):
    leaf_ids = tree.apply(X)
    explanations = [tuple(paths[leaf]) for leaf in leaf_ids]
    return explanations

explanations_sub = get_instance_explanation(tree_model.named_steps["model"], X_sub, paths)

In [ ]:
# -----------------------------
# 5️⃣ Faithfulness metrics (optimized)
# -----------------------------
def compute_consistency(explanations, predictions):
    groups = defaultdict(list)
    for exp, pred in zip(explanations, predictions):
        groups[exp].append(pred)
    consistent_groups = 0
    valid_groups = 0
    for preds in groups.values():
        if len(preds) > 1:
            valid_groups += 1
            if len(set(preds)) == 1:
                consistent_groups += 1
    return None if valid_groups == 0 else consistent_groups / valid_groups

def satisfies_explanation(x, explanation, feature_names):
    for f, op, thr in explanation:
        idx = feature_names.index(f)
        if op == "<=" and not x[idx] <= thr:
            return False
        elif op == ">" and not x[idx] > thr:
            return False
    return True

def compute_sufficiency(X, explanations, predictions, feature_names):
    suff_scores = []
    for i, explanation in enumerate(tqdm(explanations, desc="Computing sufficiency")):
        matched_preds = [
            predictions[j]
            for j in range(X.shape[0])
            if satisfies_explanation(X[j], explanation, feature_names)
        ]
        if len(matched_preds) == 0:
            continue
        ratio = matched_preds.count(predictions[i]) / len(matched_preds)
        suff_scores.append(ratio)
    return None if len(suff_scores) == 0 else float(np.mean(suff_scores))

def compute_uniqueness(explanations):
    return len(set(explanations)) / len(explanations)

## Tests with subsample = 500

In [ ]:
results = {
    "consistency": compute_consistency(explanations_sub, list(y_sub)),
    "sufficiency": compute_sufficiency(X_sub, explanations_sub, list(y_sub), feature_names),
    "uniqueness": compute_uniqueness(explanations_sub)
}

print("\nFaithfulness results (Decision Tree + leaf paths, subsample = 500, max_leaf_nodes=64):")
print(results)

Computing sufficiency: 100%|██████████| 500/500 [00:02<00:00, 168.36it/s]


Faithfulness results (Decision Tree + leaf paths, subsample = 500, max_leaf_nodes=64):
{'consistency': 1.0, 'sufficiency': 1.0, 'uniqueness': 0.094}


In [ ]:
results = {
    "consistency": compute_consistency(explanations_sub, list(y_sub)),
    "sufficiency": compute_sufficiency(X_sub, explanations_sub, list(y_sub), feature_names),
    "uniqueness": compute_uniqueness(explanations_sub)
}

print("\nFaithfulness results (Decision Tree + leaf paths, subsample = 500, max_leaf_nodes=128):")
print(results)

Computing sufficiency: 100%|██████████| 500/500 [00:02<00:00, 229.84it/s]


Faithfulness results (Decision Tree + leaf paths, subsample = 500, max_leaf_nodes=128):
{'consistency': 1.0, 'sufficiency': 1.0, 'uniqueness': 0.156}


In [ ]:
results = {
    "consistency": compute_consistency(explanations_sub, list(y_sub)),
    "sufficiency": compute_sufficiency(X_sub, explanations_sub, list(y_sub), feature_names),
    "uniqueness": compute_uniqueness(explanations_sub)
}

print("\nFaithfulness results (Decision Tree + leaf paths, subsample = 500, max_leaf_nodes=256):")
print(results)

Computing sufficiency: 100%|██████████| 500/500 [00:04<00:00, 117.64it/s]


Faithfulness results (Decision Tree + leaf paths, subsample = 500, max_leaf_nodes=256):
{'consistency': 1.0, 'sufficiency': 1.0, 'uniqueness': 0.214}


In [ ]:
results = {
    "consistency": compute_consistency(explanations_sub, list(y_sub)),
    "sufficiency": compute_sufficiency(X_sub, explanations_sub, list(y_sub), feature_names),
    "uniqueness": compute_uniqueness(explanations_sub)
}

print("\nFaithfulness results (Decision Tree + leaf paths, subsample = 500, max_leaf_nodes=512):")
print(results)

Computing sufficiency: 100%|██████████| 500/500 [00:02<00:00, 202.01it/s]


Faithfulness results (Decision Tree + leaf paths, subsample = 500, max_leaf_nodes=512):
{'consistency': 1.0, 'sufficiency': 1.0, 'uniqueness': 0.248}


In [ ]:
results = {
    "consistency": compute_consistency(explanations_sub, list(y_sub)),
    "sufficiency": compute_sufficiency(X_sub, explanations_sub, list(y_sub), feature_names),
    "uniqueness": compute_uniqueness(explanations_sub)
}

print("\nFaithfulness results (Decision Tree + leaf paths, subsample = 500, max_leaf_nodes=1024):")
print(results)

Computing sufficiency: 100%|██████████| 500/500 [00:01<00:00, 486.47it/s]


Faithfulness results (Decision Tree + leaf paths, subsample = 500, max_leaf_nodes=1024):
{'consistency': 1.0, 'sufficiency': 1.0, 'uniqueness': 0.308}


In [ ]:
results = {
    "consistency": compute_consistency(explanations_sub, list(y_sub)),
    "sufficiency": compute_sufficiency(X_sub, explanations_sub, list(y_sub), feature_names),
    "uniqueness": compute_uniqueness(explanations_sub)
}

print("\nFaithfulness results (Decision Tree + leaf paths, subsample = 500, max_leaf_nodes=2048):")
print(results)

Computing sufficiency: 100%|██████████| 500/500 [00:01<00:00, 314.67it/s]


Faithfulness results (Decision Tree + leaf paths, subsample = 500, max_leaf_nodes=2048):
{'consistency': 1.0, 'sufficiency': 1.0, 'uniqueness': 0.4}


# Logistic Regression + Top-k coefficients

In [ ]:
from sklearn.linear_model import LogisticRegression

# Pipeline com pré-processamento igual ao Random Forest
logreg_model = Pipeline([
    ("preprocessing", preprocessor),
    ("model", LogisticRegression(max_iter=500, solver="liblinear"))
])

logreg_model.fit(X_train, y_train)

# Transformar X_test para métricas
X_test_trans = logreg_model.named_steps["preprocessing"].transform(X_test)
y_pred_test = logreg_model.predict(X_test)

In [ ]:
def topk_coeff_explanation(model, X_transformed, feature_names, k=3):
    """
    Para cada instância, seleciona as k features com maior |coef * valor da feature|
    Retorna uma lista de tuplos: (feature_name, sinal do coeficiente)
    """
    coef = model.named_steps["model"].coef_[0]  # coeficientes da classe positiva
    explanations = []
    for x in X_transformed:
        impacts = np.abs(coef * x)  # magnitude da contribuição
        topk_idx = impacts.argsort()[-k:]  # índices das top-k
        explanation = [(feature_names[i], int(np.sign(coef[i]))) for i in topk_idx]
        explanations.append(tuple(sorted(explanation)))  # tupla para métricas
    return explanations

# Gerar explicações
k = 3
logreg_explanations = topk_coeff_explanation(logreg_model, X_test_trans, feature_names, k)

In [ ]:
# Helper function to check if an instance satisfies a Logistic Regression explanation
def satisfies_explanation_logreg(x_instance_transformed, explanation, transformed_feature_names):
    """
    Check whether a transformed instance satisfies a discretized explanation
    from Logistic Regression (feature name, sign of coefficient).
    """
    for feature_name, sign in explanation:
        try:
            idx = transformed_feature_names.index(feature_name)
            # Check the sign of the feature value in the instance
            if int(np.sign(x_instance_transformed[idx])) != sign:
                return False
        except ValueError:
            # Feature name not found in transformed_feature_names (shouldn't happen if feature_names are consistent)
            return False
    return True

# Helper function to compute sufficiency for Logistic Regression explanations
def compute_sufficiency_logreg(X_transformed, explanations, predictions, transformed_feature_names):
    """
    Sufficiency measures whether an explanation is sufficient
    to guarantee the model prediction on other instances for Logistic Regression.
    """
    suff_scores = []

    for i, explanation_i in enumerate(tqdm(explanations, desc="Computing sufficiency")):
        matched_predictions = [
            predictions[j]
            for j in range(len(X_transformed))
            if satisfies_explanation_logreg(X_transformed[j], explanation_i, transformed_feature_names)
        ]

        if len(matched_predictions) == 0:
            continue

        same_prediction_ratio = (
            matched_predictions.count(predictions[i]) / len(matched_predictions)
        )

        suff_scores.append(same_prediction_ratio)

    if len(suff_scores) == 0:
        return None

    return float(np.mean(suff_scores))


# Reusar funções que já tens
consistency = compute_consistency(logreg_explanations, y_pred_test)
sufficiency = compute_sufficiency_logreg(X_test_trans, logreg_explanations, y_pred_test, feature_names)
uniqueness = len(set(logreg_explanations)) / len(logreg_explanations)

Streaming output truncated to the last 5000 lines.



Computing sufficiency:  63%|██████▎   | 5712/9045 [03:47<01:57, 28.44it/s]



Computing sufficiency:  63%|██████▎   | 5715/9045 [03:47<01:58, 28.14it/s]



Computing sufficiency:  63%|██████▎   | 5718/9045 [03:47<02:02, 27.07it/s]



Computing sufficiency:  63%|██████▎   | 5721/9045 [03:47<02:00, 27.48it/s]



Computing sufficiency:  63%|██████▎   | 5724/9045 [03:47<02:00, 27.63it/s]



Computing sufficiency:  63%|██████▎   | 5727/9045 [03:48<01:59, 27.73it/s]



Computing sufficiency:  63%|██████▎   | 5730/9045 [03:48<02:00, 27.60it/s]



Computing sufficiency:  63%|██████▎   | 5733/9045 [03:48<01:59, 27.67it/s]



Computing sufficiency:  63%|██████▎   | 5736/9045 [03:48<01:59, 27.67it/s]



Computing sufficiency:  63%|██████▎   | 5739/9045 [03:48<01:59, 27.73it/s]



Computing sufficiency:  63%|██████▎   | 5742/9045 [03:48<01:57, 28.09it/s]



Computing sufficiency:  64%|██████▎   | 5745/9045 [03:48<02:02, 26.97it/s]



Computing 


Faithfulness metrics (Logistic Regression + Top-k coefficients):
Consistency: 0.25
Sufficiency: None
Uniqueness: 0.0008844665561083472


In [94]:
print("\nFaithfulness metrics (Logistic Regression + Top-k coefficients):")
print(f"Consistency: {consistency}")
print(f"Sufficiency: {sufficiency}")
print(f"Uniqueness: {uniqueness}")


Faithfulness metrics (Logistic Regression + Top-k coefficients):
Consistency: 0.25
Sufficiency: None
Uniqueness: 0.0008844665561083472


# Random Forest + SHAP

## Random Forest

In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Load dataset
df = pd.read_csv("/content/drive/MyDrive/IACD/3 ANO/IC/adult.csv")

TARGET_COL = "income"


# Basic cleaning
# Remove rows with missing values represented as '?'
df = df.replace("?", np.nan).dropna()

X = df.drop(columns=TARGET_COL)
y = df[TARGET_COL]


# Feature types
categorical_features = X.select_dtypes(include="object").columns.tolist()
numerical_features = X.select_dtypes(exclude="object").columns.tolist()


# Preprocessing
# One-hot encode categorical variables, keep numerical as-is
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
        ("num", "passthrough", numerical_features)
    ],
    remainder="drop",
    sparse_threshold=0.3,
)


# Model
rf_model = RandomForestClassifier(
    n_estimators=50,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)


# Pipeline
pipeline = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", rf_model)
    ]
)


# Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


# Training
pipeline.fit(X_train, y_train)


# Evaluation (predictions + metrics)
y_pred = pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.8479823106688779

Classification Report:
               precision    recall  f1-score   support

       <=50K       0.88      0.93      0.90      6803
        >50K       0.73      0.61      0.67      2242

    accuracy                           0.85      9045
   macro avg       0.80      0.77      0.78      9045
weighted avg       0.84      0.85      0.84      9045


Confusion Matrix:
 [[6297  506]
 [ 869 1373]]


In [ ]:
# Prepare XAI artefacts

preprocessor_trained = pipeline.named_steps["preprocessing"]

X_test_transformed = preprocessor_trained.transform(X_test)

try:
    cat_transformer = preprocessor_trained.named_transformers_["cat"]
    cat_feature_names = cat_transformer.get_feature_names_out(categorical_features)
except Exception:
    cat_feature_names = []
    try:
        categories = preprocessor_trained.named_transformers_["cat"].categories_
        for feat, cats in zip(categorical_features, categories):
            cat_feature_names.extend([f"{feat}_{val}" for val in cats])
    except Exception:
        cat_feature_names = categorical_features.copy()

num_feature_names = numerical_features.copy()

feature_names = list(cat_feature_names) + list(num_feature_names)

n_transformed_cols = X_test_transformed.shape[1]

if len(feature_names) != n_transformed_cols:
    print("Warning: feature_names length differs from transformed columns.")
    print(f"len(feature_names)={len(feature_names)}, transformed_cols={n_transformed_cols}")
    feature_names = [f"f_{i}" for i in range(n_transformed_cols)]
    print("Fallback: created generic feature names to match transformed shape.")

print("\nSanity checks:")
print("X_test shape:", X_test.shape)
print("X_test_transformed shape:", X_test_transformed.shape)
print("Number of feature_names:", len(feature_names))


Sanity checks:
X_test shape: (9045, 14)
X_test_transformed shape: (9045, 104)
Number of feature_names: 104


## SHAP

In [ ]:
import numpy as np
from collections import defaultdict
import shap

In [ ]:
def compute_shap_values(model, X):
    """
    Compute SHAP values for a tree-based model.
    Returns SHAP values for the positive class.
    """
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)

    # Binary classification: use positive class
    return shap_values[1]

In [ ]:
def discretize_shap_topk(shap_row, feature_names, k=5):
    """
    Compress a SHAP explanation by keeping only the Top-K
    features with highest absolute contribution and their sign.
    """
    topk_idx = np.argsort(np.abs(shap_row))[-k:]

    explanation = [
        (feature_names[i], int(np.sign(shap_row[i])))
        for i in topk_idx
        if shap_row[i] != 0
    ]

    return tuple(sorted(explanation))

In [ ]:
def discretize_explanations(shap_values, feature_names, k=5):
    """
    Apply discretization to all SHAP explanations.
    """
    return [
        discretize_shap_topk(shap_values[i], feature_names, k)
        for i in range(len(shap_values))
    ]

In [ ]:
def compute_consistency(explanations, predictions):
    """
    Consistency measures whether instances sharing the same
    explanation also share the same prediction.
    """
    groups = defaultdict(list)

    for exp, pred in zip(explanations, predictions):
        groups[exp].append(pred)

    consistent_groups = 0
    valid_groups = 0

    for preds in groups.values():
        if len(preds) > 1:
            valid_groups += 1
            if len(set(preds)) == 1:
                consistent_groups += 1

    if valid_groups == 0:
        return None

    return consistent_groups / valid_groups

In [ ]:
def satisfies_explanation(x, explanation, feature_names):
    """
    Check whether a transformed instance satisfies
    a discretized explanation.
    """
    for feature, sign in explanation:
        idx = feature_names.index(feature)
        if int(np.sign(x[idx])) != sign:
            return False
    return True

In [ ]:
def compute_sufficiency(X, explanations, predictions, feature_names):
    """
    Sufficiency measures whether an explanation is sufficient
    to guarantee the model prediction on other instances.
    """
    sufficiency_scores = []

    # Adiciona tqdm para monitorar o progresso
    for i, explanation in enumerate(tqdm(explanations, desc="Computing sufficiency")):
        matched_predictions = [
            predictions[j]
            for j in range(len(X))
            if satisfies_explanation(X[j], explanation, feature_names)
        ]

        if len(matched_predictions) == 0:
            continue

        same_prediction_ratio = (
            matched_predictions.count(predictions[i]) / len(matched_predictions)
        )

        sufficiency_scores.append(same_prediction_ratio)

    if len(sufficiency_scores) == 0:
        return None

    return float(np.mean(sufficiency_scores))

In [ ]:
def compute_uniqueness(explanations):
    """
    Uniqueness is the proportion of unique explanations
    in the evaluation set.
    """
    return len(set(explanations)) / len(explanations)

In [ ]:
def evaluate_faithfulness_shap(
    model,
    X_test_transformed,
    y_pred,
    feature_names,
    k=5
):
    """
    Full faithfulness evaluation pipeline for SHAP explanations.
    """
    shap_values = compute_shap_values(model, X_test_transformed)

    explanations = discretize_explanations(
        shap_values,
        feature_names,
        k=k
    )

    results = {
        "consistency": compute_consistency(explanations, list(y_pred)),
        "sufficiency": compute_sufficiency(
            X_test_transformed,
            explanations,
            list(y_pred),
            feature_names
        ),
        "uniqueness": compute_uniqueness(explanations)
    }

    return results

In [ ]:
# Seed para reproducibilidade
np.random.seed(42)

# Subsample 100 instâncias (ou menos se quiseres mais rápido)
sample_size = 100
idx = np.random.choice(len(X_test_transformed), size=sample_size, replace=False)

X_xai = X_test_transformed[idx]
y_xai = y_pred[idx]

In [ ]:
results = evaluate_faithfulness_shap(
    model=pipeline.named_steps["model"],
    X_test_transformed=X_xai,
    y_pred=y_xai,
    feature_names=feature_names,
    k=3
)

print(results)

Computing sufficiency: 100%|██████████| 104/104 [00:00<00:00, 948.67it/s]

{'consistency': 0.0, 'sufficiency': None, 'uniqueness': 0.019230769230769232}


# Random Forest + LIME

In [96]:
!pip install lime --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [97]:
# -----------------------------
# 1️⃣ Imports
# -----------------------------
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from collections import defaultdict
from tqdm import tqdm
from lime.lime_tabular import LimeTabularExplainer

In [98]:
# -----------------------------
# 2️⃣ Load and preprocess dataset
# -----------------------------
df = pd.read_csv("/content/drive/MyDrive/IACD/3 ANO/IC/adult.csv")
TARGET_COL = "income"
df = df.replace("?", np.nan).dropna()

X = df.drop(columns=TARGET_COL)
y = df[TARGET_COL]

categorical_features = X.select_dtypes(include="object").columns.tolist()
numerical_features = X.select_dtypes(exclude="object").columns.tolist()

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
    ("num", "passthrough", numerical_features)
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [99]:
# -----------------------------
# 3️⃣ Train Random Forest
# -----------------------------
rf_model = Pipeline([
    ("preprocessing", preprocessor),
    ("model", RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1))
])
rf_model.fit(X_train, y_train)

# Transform test set for LIME
X_test_trans = rf_model.named_steps["preprocessing"].transform(X_test)
feature_names_trans = list(rf_model.named_steps["preprocessing"].get_feature_names_out())

In [100]:
# -----------------------------
# 4️⃣ Setup LIME Explainer
# -----------------------------
# LimeTabular expects the raw preprocessed numpy array, plus categorical features indices
cat_idx = [i for i, f in enumerate(feature_names_trans) if any(f.startswith(c) for c in categorical_features)]

explainer = LimeTabularExplainer(
    training_data=rf_model.named_steps["preprocessing"].transform(X_train),
    feature_names=feature_names_trans,
    class_names=[str(c) for c in rf_model.named_steps["model"].classes_],
    categorical_features=cat_idx,
    discretize_continuous=True,
    random_state=42
)

In [101]:
# -----------------------------
# 5️⃣ Generate Top-k LIME explanations
# -----------------------------
k = 3  # Top-k features

def lime_topk_explanation(instance):
    exp = explainer.explain_instance(
        instance,
        rf_model.named_steps["model"].predict_proba,
        num_features=k
    )
    # Retorna apenas os nomes das features top-k
    return tuple(sorted([feat for feat, weight in exp.as_list()]))

# Subsample para acelerar
sample_size = 100
np.random.seed(42)
idx = np.random.choice(X_test_trans.shape[0], size=sample_size, replace=False)
X_sub_trans = X_test_trans[idx]
y_sub_pred = rf_model.predict(X_test.iloc[idx])

lime_explanations = [lime_topk_explanation(X_sub_trans[i]) for i in tqdm(range(sample_size), desc="Generating LIME explanations")]





Generating LIME explanations:   0%|          | 0/100 [00:00<?, ?it/s]



Generating LIME explanations:   1%|          | 1/100 [00:02<03:46,  2.29s/it]



Generating LIME explanations:   2%|▏         | 2/100 [00:05<04:25,  2.71s/it]



Generating LIME explanations:   3%|▎         | 3/100 [00:09<05:48,  3.59s/it]



Generating LIME explanations:   4%|▍         | 4/100 [00:12<04:49,  3.01s/it]



Generating LIME explanations:   5%|▌         | 5/100 [00:14<04:31,  2.86s/it]



Generating LIME explanations:   6%|▌         | 6/100 [00:17<04:17,  2.74s/it]



Generating LIME explanations:   7%|▋         | 7/100 [00:19<04:12,  2.72s/it]



Generating LIME explanations:   8%|▊         | 8/100 [00:25<05:33,  3.62s/it]



Generating LIME explanations:   9%|▉         | 9/100 [00:28<05:06,  3.37s/it]



Generating LIME explanations:  10%|█         | 10/100 [00:31<04:52,  3.25s/it]



Generating LIME explanations:  11%|█         | 11/100 [00:33<04:33,  3.08s/it]



Generating LIME explanations: 

In [102]:
# -----------------------------
# 6️⃣ Faithfulness metrics
# -----------------------------
def compute_consistency(explanations, predictions):
    groups = defaultdict(list)
    for exp, pred in zip(explanations, predictions):
        groups[exp].append(pred)
    consistent_groups = 0
    valid_groups = 0
    for preds in groups.values():
        if len(preds) > 1:
            valid_groups += 1
            if len(set(preds)) == 1:
                consistent_groups += 1
    return None if valid_groups == 0 else consistent_groups / valid_groups

def compute_sufficiency(X_trans, explanations, predictions):
    suff_scores = []
    for i, exp in enumerate(tqdm(explanations, desc="Computing sufficiency")):
        matched_preds = [
            predictions[j] for j in range(len(X_trans))
            if all(X_trans[j][feature_names_trans.index(f)] != 0 for f in exp if f in feature_names_trans)
        ]
        if len(matched_preds) == 0:
            continue
        ratio = matched_preds.count(predictions[i]) / len(matched_preds)
        suff_scores.append(ratio)
    return None if len(suff_scores) == 0 else float(np.mean(suff_scores))

def compute_uniqueness(explanations):
    return len(set(explanations)) / len(explanations)

In [103]:
# -----------------------------
# 7️⃣ Calculate metrics
# -----------------------------
consistency = compute_consistency(lime_explanations, y_sub_pred)
sufficiency = compute_sufficiency(X_sub_trans, lime_explanations, y_sub_pred)
uniqueness = compute_uniqueness(lime_explanations)

print("\nFaithfulness metrics (Random Forest + LIME, Top-3 features, subsample={}):".format(sample_size))
print(f"Consistency: {consistency}")
print(f"Sufficiency: {sufficiency}")
print(f"Uniqueness: {uniqueness}")





Computing sufficiency: 100%|██████████| 100/100 [00:00<00:00, 1403.45it/s]


Faithfulness metrics (Random Forest + LIME, Top-3 features, subsample=100):
Consistency: 0.5454545454545454
Sufficiency: 0.68
Uniqueness: 0.15


# Random Forest + Anchors

In [ ]:
!pip install alibi --quiet

In [ ]:
# -----------------------------
# 1️⃣ Imports
# -----------------------------
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from collections import defaultdict
from tqdm import tqdm
from alibi.explainers import AnchorTabular

In [ ]:
# -----------------------------
# 2️⃣ Carregar dataset Adult
# -----------------------------
df = pd.read_csv("/content/drive/MyDrive/IACD/3 ANO/IC/adult.csv")

TARGET_COL = "income"
df = df.replace("?", np.nan).dropna()

X = df.drop(columns=TARGET_COL)
y = df[TARGET_COL]

categorical_features = X.select_dtypes(include="object").columns.tolist()
numerical_features = X.select_dtypes(exclude="object").columns.tolist()

In [ ]:
# -----------------------------
# 3️⃣ Preprocessing e Train/Test split
# -----------------------------
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
    ("num", "passthrough", numerical_features)
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# -----------------------------
# 4️⃣ Treinar Random Forest
# -----------------------------
rf_model = Pipeline([
    ("preprocessing", preprocessor),
    ("model", RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1))
])

rf_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'gender',
                                                   'native-country']),
                                                 ('num', 'passthrough',
                                                  ['age', 'fnlwgt',
                                                   'educational-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week'])])),
                ('model',
                 RandomForestClassifier(n_estimators=50, n_jobs=-1,
                                        random_state=42))])

In [ ]:
# -----------------------------
# 5️⃣ Preparar feature names e categorical map para Anchors
# -----------------------------
# Feature names pós-transformação
cat_transformer = rf_model.named_steps["preprocessing"].named_transformers_["cat"]
cat_names = cat_transformer.get_feature_names_out(categorical_features)
feature_names = list(cat_names) + numerical_features
feature_name_to_idx = {
    name: i for i, name in enumerate(feature_names)
}

# Map para Anchors: índice da coluna → categorias possíveis
categorical_map = {}
start_idx = 0
for cats in cat_transformer.categories_:
    for _ in range(len(cats)):
        categorical_map[start_idx] = list(cats)
        start_idx += 1

In [ ]:
# -----------------------------
# 6️⃣ Função para Anchors (necessário callable)
# -----------------------------
def predict_fn(X_transformed):
    """
    X_transformed: já pré-processado
    Retorna rótulos preditos
    """
    return rf_model.named_steps["model"].predict(X_transformed)

In [ ]:
# -----------------------------
# 7️⃣ Criar explicador AnchorTabular
# -----------------------------
explainer = AnchorTabular(
    predictor=predict_fn,
    feature_names=feature_names,
    categorical_names=categorical_map
)

# Ajustar distribuições para numéricos
X_train_trans = rf_model.named_steps["preprocessing"].transform(X_train)
explainer.fit(X_train_trans, disc_perc=[25,50,75])  # bins para features numéricas

AnchorTabular(meta={
  'name': 'AnchorTabular',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'seed': None, 'disc_perc': [25, 50, 75]},
  'version': '0.9.6'}
)

In [ ]:
# -----------------------------
# 8️⃣ Subsample de teste para acelerar experimentos
# -----------------------------
np.random.seed(42)
sample_size = 200  # ajusta para mais rápido
idx = np.random.choice(X_test.shape[0], size=sample_size, replace=False)
X_sub = X_test.iloc[idx]
y_sub = y_test.iloc[idx]

# Pré-processar
X_sub_trans = rf_model.named_steps["preprocessing"].transform(X_sub)

In [ ]:
# -----------------------------
# 9️⃣ Funções de métricas de fidelidade
# -----------------------------
def anchor_to_indices(anchor, feature_name_to_idx):
    idxs = []
    for cond in anchor:
        if cond in feature_name_to_idx:
            idxs.append(feature_name_to_idx[cond])
    return idxs


def satisfies_anchor(x, x_ref, anchor, feature_name_to_idx):
    idxs = anchor_to_indices(anchor, feature_name_to_idx)
    for idx in idxs:
        if x[idx] != x_ref[idx]:
            return False
    return True


def compute_sufficiency(X, anchors_list, predictions):
    suff_scores = []
    for i, anchor in enumerate(tqdm(anchors_list, desc="Computing sufficiency")):
        matched_preds = [
            predictions[j]
            for j in range(len(X))
            if satisfies_anchor(X[j], X[i], anchor)
        ]
        if len(matched_preds) == 0:
            continue
        ratio = matched_preds.count(predictions[i]) / len(matched_preds)
        suff_scores.append(ratio)
    return None if len(suff_scores) == 0 else float(np.mean(suff_scores))


def compute_sufficiency(X, anchors_list, predictions, feature_name_to_idx):
    suff_scores = []

    for i, anchor in enumerate(tqdm(anchors_list, desc="Computing sufficiency")):
        matched_preds = [
            predictions[j]
            for j in range(len(X))
            if satisfies_anchor(X[j], X[i], anchor, feature_name_to_idx)
        ]

        if len(matched_preds) == 0:
            continue

        ratio = matched_preds.count(predictions[i]) / len(matched_preds)
        suff_scores.append(ratio)

    return None if len(suff_scores) == 0 else float(np.mean(suff_scores))

## threshold = 0.5

In [ ]:
# -----------------------------
# 🔟 Gerar anchors para cada instância do subsample
# -----------------------------
sample_size = 50        # número de instâncias a explicar
threshold = 0.5         # menor threshold = menos combinações testadas
# top_k_features = None  # removido, usar todas as features

# Subsample do dataset
np.random.seed(42)
idx = np.random.choice(X_sub_trans.shape[0], size=sample_size, replace=False)
X_sub = X_sub_trans[idx]

def explain_instance(x):
    return explainer.explain(x.reshape(1,-1), threshold=threshold).anchor


anchors_list = Parallel(n_jobs=-1)(
    delayed(explain_instance)(x) for x in tqdm(X_sub, desc="Generating Anchors")
)

print(f"Generated {len(anchors_list)} anchors for {sample_size} instances")





Generating Anchors:   0%|          | 0/50 [00:00<?, ?it/s]



Generating Anchors:   4%|▍         | 2/50 [00:00<00:03, 13.23it/s]/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(




Generating Anchors:   4%|▍         | 2/50 [00:17<00:03, 13.23it/s]



Generating Anchors:   8%|▊         | 4/50 [00:39<08:56, 11.66s/it]



Generating Anchors:  12%|█▏        | 6/50 [01:11<10:01, 13.66s/it]



Generating Anchors:  16%|█▌        | 8/50 [01:13<06:02,  8.62s/it]



Generating Anchors:  20%|██        | 10/50 [01:14<03:48,  5.72s/it]



Generating Anchors:  24%|██▍       | 12/50 [01:18<02:45,  4.36s/it]



Generating Anchors:  28%|██▊       | 14/50 [01:20<02:01,  3.37s/it]



Generating Anchors:  32%|███▏      | 16/50 [01:37<02:46,  4.90s/it]



Generating Anchors:  36%|███▌      | 18/50 [01:59<03

Generated 50 anchors for 50 instances


In [ ]:
# -----------------------------
# 1️⃣1️⃣ Calcular métricas
# -----------------------------

X_eval = X_sub          # dados transformados usados no Anchor
y_eval = y_pred_sub     # predições correspondentes

y_pred_sub = rf_model.named_steps["model"].predict(X_sub)

consistency = compute_consistency(anchors_list, y_eval)
sufficiency = compute_sufficiency(X_eval, anchors_list, y_eval, feature_name_to_idx)
uniqueness = len(set(map(tuple, anchors_list))) / len(anchors_list)

print("\nFaithfulness metrics (Anchors, Random Forest, threshold = 0.5, subsample={}):".format(sample_size))
print(f"Consistency: {consistency}")
print(f"Sufficiency: {sufficiency}")
print(f"Uniqueness: {uniqueness}")





Computing sufficiency: 100%|██████████| 50/50 [00:00<00:00, 9702.75it/s]


Faithfulness metrics (Anchors, Random Forest, threshold = 0.5, subsample=50):
Consistency: 1.0
Sufficiency: 0.58
Uniqueness: 0.22


## threshold = 0.6

In [ ]:
# -----------------------------
# 🔟 Gerar anchors para cada instância do subsample
# -----------------------------
sample_size = 50        # número de instâncias a explicar
threshold = 0.6         # menor threshold = menos combinações testadas
# top_k_features = None  # removido, usar todas as features

# Subsample do dataset
np.random.seed(42)
idx = np.random.choice(X_sub_trans.shape[0], size=sample_size, replace=False)
X_sub = X_sub_trans[idx]

def explain_instance(x):
    return explainer.explain(x.reshape(1,-1), threshold=threshold).anchor


anchors_list = Parallel(n_jobs=-1)(
    delayed(explain_instance)(x) for x in tqdm(X_sub, desc="Generating Anchors")
)

print(f"Generated {len(anchors_list)} anchors for {sample_size} instances")





Generating Anchors:   0%|          | 0/50 [00:00<?, ?it/s]



Generating Anchors:   8%|▊         | 4/50 [00:00<00:09,  4.85it/s]



Generating Anchors:  12%|█▏        | 6/50 [00:40<06:05,  8.31s/it]



Generating Anchors:  16%|█▌        | 8/50 [00:41<03:51,  5.52s/it]



Generating Anchors:  20%|██        | 10/50 [00:43<02:34,  3.86s/it]



Generating Anchors:  24%|██▍       | 12/50 [00:46<01:56,  3.07s/it]



Generating Anchors:  28%|██▊       | 14/50 [00:47<01:25,  2.36s/it]



Generating Anchors:  32%|███▏      | 16/50 [00:49<01:00,  1.79s/it]



Generating Anchors:  36%|███▌      | 18/50 [01:23<03:29,  6.54s/it]



Generating Anchors:  40%|████      | 20/50 [02:56<09:25, 18.86s/it]/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(




Generating Anchors:  44%|████▍     | 22/50 [03:41<

Generated 50 anchors for 50 instances


In [ ]:
# -----------------------------
# 1️⃣1️⃣ Calcular métricas
# -----------------------------

X_eval = X_sub          # dados transformados usados no Anchor
y_eval = y_pred_sub     # predições correspondentes

y_pred_sub = rf_model.named_steps["model"].predict(X_sub)

consistency = compute_consistency(anchors_list, y_eval)
sufficiency = compute_sufficiency(X_eval, anchors_list, y_eval, feature_name_to_idx)
uniqueness = len(set(map(tuple, anchors_list))) / len(anchors_list)

print("\nFaithfulness metrics (Anchors, Random Forest, threshold = 0.6, subsample={}):".format(sample_size))
print(f"Consistency: {consistency}")
print(f"Sufficiency: {sufficiency}")
print(f"Uniqueness: {uniqueness}")





Computing sufficiency: 100%|██████████| 50/50 [00:00<00:00, 23278.41it/s]


Faithfulness metrics (Anchors, Random Forest, threshold = 0.6, subsample=50):
Consistency: 1.0
Sufficiency: 0.58
Uniqueness: 0.32


## threshold = 0.7

In [ ]:
# -----------------------------
# 🔟 Gerar anchors para cada instância do subsample
# -----------------------------
sample_size = 50        # número de instâncias a explicar
threshold = 0.7         # menor threshold = menos combinações testadas
# top_k_features = None  # removido, usar todas as features

# Subsample do dataset
np.random.seed(42)
idx = np.random.choice(X_sub_trans.shape[0], size=sample_size, replace=False)
X_sub = X_sub_trans[idx]

def explain_instance(x):
    return explainer.explain(x.reshape(1,-1), threshold=threshold).anchor


anchors_list = Parallel(n_jobs=-1)(
    delayed(explain_instance)(x) for x in tqdm(X_sub, desc="Generating Anchors")
)

print(f"Generated {len(anchors_list)} anchors for {sample_size} instances")





Generating Anchors:   0%|          | 0/50 [00:00<?, ?it/s]



Generating Anchors:   8%|▊         | 4/50 [00:02<00:24,  1.85it/s]



Generating Anchors:  12%|█▏        | 6/50 [00:34<05:08,  7.01s/it]



Generating Anchors:  16%|█▌        | 8/50 [01:00<06:30,  9.31s/it]



Generating Anchors:  20%|██        | 10/50 [01:05<04:36,  6.92s/it]



Generating Anchors:  24%|██▍       | 12/50 [01:14<03:53,  6.14s/it]



Generating Anchors:  28%|██▊       | 14/50 [01:22<03:18,  5.51s/it]



Generating Anchors:  32%|███▏      | 16/50 [03:39<14:19, 25.27s/it]/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(




Generating Anchors:  36%|███▌      | 18/50 [04:37<14:05, 26.44s/it]



Generating Anchors:  40%|████      | 20/50 [05:20<12:28, 24.96s/it]



Generating Anchors:  44%|████▍     | 22/50 [05:25<

Generated 50 anchors for 50 instances


In [ ]:
# -----------------------------
# 1️⃣1️⃣ Calcular métricas
# -----------------------------

X_eval = X_sub          # dados transformados usados no Anchor
y_eval = y_pred_sub     # predições correspondentes

y_pred_sub = rf_model.named_steps["model"].predict(X_sub)

consistency = compute_consistency(anchors_list, y_eval)
sufficiency = compute_sufficiency(X_eval, anchors_list, y_eval, feature_name_to_idx)
uniqueness = len(set(map(tuple, anchors_list))) / len(anchors_list)

print("\nFaithfulness metrics (Anchors, Random Forest, threshold = 0.7, subsample={}):".format(sample_size))
print(f"Consistency: {consistency}")
print(f"Sufficiency: {sufficiency}")
print(f"Uniqueness: {uniqueness}")





Computing sufficiency: 100%|██████████| 50/50 [00:00<00:00, 16723.70it/s]


Faithfulness metrics (Anchors, Random Forest, threshold = 0.7, subsample=50):
Consistency: 1.0
Sufficiency: 0.58
Uniqueness: 0.44


## threshold = 0.8

In [ ]:
# -----------------------------
# 🔟 Gerar anchors para cada instância do subsample
# -----------------------------
sample_size = 50        # número de instâncias a explicar
threshold = 0.8         # menor threshold = menos combinações testadas
# top_k_features = None  # removido, usar todas as features

# Subsample do dataset
np.random.seed(42)
idx = np.random.choice(X_sub_trans.shape[0], size=sample_size, replace=False)
X_sub = X_sub_trans[idx]

def explain_instance(x):
    return explainer.explain(x.reshape(1,-1), threshold=threshold).anchor


anchors_list = Parallel(n_jobs=-1)(
    delayed(explain_instance)(x) for x in tqdm(X_sub, desc="Generating Anchors")
)

print(f"Generated {len(anchors_list)} anchors for {sample_size} instances")





Generating Anchors:   0%|          | 0/50 [00:00<?, ?it/s]



Generating Anchors:   4%|▍         | 2/50 [00:00<00:05,  8.30it/s]



Generating Anchors:   8%|▊         | 4/50 [00:48<10:54, 14.23s/it]



Generating Anchors:  12%|█▏        | 6/50 [01:56<17:07, 23.36s/it]



Generating Anchors:  16%|█▌        | 8/50 [02:10<11:46, 16.82s/it]



Generating Anchors:  20%|██        | 10/50 [02:17<07:59, 11.98s/it]



Generating Anchors:  24%|██▍       | 12/50 [02:30<06:22, 10.07s/it]



Generating Anchors:  28%|██▊       | 14/50 [02:39<04:58,  8.30s/it]



Generating Anchors:  32%|███▏      | 16/50 [05:38<19:19, 34.11s/it]/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(




Generating Anchors:  36%|███▌      | 18/50 [07:12<20:24, 38.28s/it]



Generating Anchors:  40%|████      | 20/50 [09:00<2

Generated 50 anchors for 50 instances


In [ ]:
# -----------------------------
# 1️⃣1️⃣ Calcular métricas
# -----------------------------

X_eval = X_sub          # dados transformados usados no Anchor
y_eval = y_pred_sub     # predições correspondentes

y_pred_sub = rf_model.named_steps["model"].predict(X_sub)

consistency = compute_consistency(anchors_list, y_eval)
sufficiency = compute_sufficiency(X_eval, anchors_list, y_eval, feature_name_to_idx)
uniqueness = len(set(map(tuple, anchors_list))) / len(anchors_list)

print("\nFaithfulness metrics (Anchors, Random Forest, threshold = 0.8, subsample={}):".format(sample_size))
print(f"Consistency: {consistency}")
print(f"Sufficiency: {sufficiency}")
print(f"Uniqueness: {uniqueness}")





Computing sufficiency: 100%|██████████| 50/50 [00:00<00:00, 13883.83it/s]


Faithfulness metrics (Anchors, Random Forest, threshold = 0.8, subsample=50):
Consistency: 1.0
Sufficiency: 0.58
Uniqueness: 0.44


## threshold = 0.9

In [ ]:
# -----------------------------
# 🔟 Gerar anchors para cada instância do subsample
# -----------------------------
sample_size = 50        # número de instâncias a explicar
threshold = 0.9         # menor threshold = menos combinações testadas
# top_k_features = None  # removido, usar todas as features

# Subsample do dataset
np.random.seed(42)
idx = np.random.choice(X_sub_trans.shape[0], size=sample_size, replace=False)
X_sub = X_sub_trans[idx]

def explain_instance(x):
    return explainer.explain(x.reshape(1,-1), threshold=threshold).anchor


anchors_list = Parallel(n_jobs=-1)(
    delayed(explain_instance)(x) for x in tqdm(X_sub, desc="Generating Anchors")
)

print(f"Generated {len(anchors_list)} anchors for {sample_size} instances")





Generating Anchors:   0%|          | 0/50 [00:00<?, ?it/s]



Generating Anchors:   8%|▊         | 4/50 [00:02<00:27,  1.68it/s]



Generating Anchors:  12%|█▏        | 6/50 [01:05<09:50, 13.42s/it]



Generating Anchors:  16%|█▌        | 8/50 [01:15<07:16, 10.40s/it]/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(




Generating Anchors:  20%|██        | 10/50 [01:54<09:01, 13.53s/it]



Generating Anchors:  24%|██▍       | 12/50 [02:17<08:08, 12.85s/it]



Generating Anchors:  28%|██▊       | 14/50 [02:25<05:58,  9.95s/it]



Generating Anchors:  32%|███▏      | 16/50 [03:08<07:43, 13.64s/it]



Generating Anchors:  36%|███▌      | 18/50 [04:59<14:13, 26.66s/it]



Generating Anchors:  40%|████      | 20/50 [10:13<33:18, 66.61s/it]



Generating Anchors:  44%|████▍     | 22/50 [10:55<

Generated 50 anchors for 50 instances


In [ ]:
# -----------------------------
# 1️⃣1️⃣ Calcular métricas
# -----------------------------

X_eval = X_sub          # dados transformados usados no Anchor
y_eval = y_pred_sub     # predições correspondentes

y_pred_sub = rf_model.named_steps["model"].predict(X_sub)

consistency = compute_consistency(anchors_list, y_eval)
sufficiency = compute_sufficiency(X_eval, anchors_list, y_eval, feature_name_to_idx)
uniqueness = len(set(map(tuple, anchors_list))) / len(anchors_list)

print("\nFaithfulness metrics (Anchors, Random Forest, threshold = 0.9, subsample={}):".format(sample_size))
print(f"Consistency: {consistency}")
print(f"Sufficiency: {sufficiency}")
print(f"Uniqueness: {uniqueness}")





Computing sufficiency: 100%|██████████| 50/50 [00:00<00:00, 7737.43it/s]


Faithfulness metrics (Anchors, Random Forest, threshold = 0.9, subsample=50):
Consistency: 1.0
Sufficiency: 0.58
Uniqueness: 0.7
